In [1]:
import sentence_transformers  # dependency for sentence_feature_extractor.py
# check that we can use sentence_transformers
print(sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2').encode(['hello world']).shape)
# import openai  # dependency for chatgpt.py
import evadb
cursor = evadb.connect(evadb_dir=("./eva_db")).cursor()

/home/oscar/.cache/pypoetry/virtualenvs/words-Cdd64V6x-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/oscar/.cache/pypoetry/virtualenvs/words-Cdd64V6x-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


(1, 384)


In [2]:
# Register the function as embed text
cursor.query("""
DROP FUNCTION IF EXISTS EmbedText
""").df()
cursor.query("""
CREATE FUNCTION EmbedText IMPL './sentence_feature_extractor.py'
""").df()
cursor.query("""
SELECT EmbedText('hello world')
""").df()

11-19-2023 00:19:15 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_function:0086] Function EmbedText does not exist, therefore cannot be dropped.


,features
0,"[[-0.03447732, 0.031023178, 0.0067349477, 0.02..."


In [4]:
cursor.query("""
DROP TABLE IF EXISTS words
""").df()
cursor.query("""
CREATE TABLE words (
            word TEXT(0)
            )"""
).df()
cursor.query("""
SELECT * FROM words
""").df()


""


In [ ]:
# iterate thru ./top_english_words_lower_50000.txt

from pathlib import Path
full_text = Path('./top_english_words_lower_50000.txt').read_text()
words = full_text.split('\n')
print(words[:10])

from tqdm import tqdm   

for word in tqdm(words):
    bad_chars = '{}-;:"\''
    bad_chars_translation = str.maketrans(dict.fromkeys(bad_chars, ''))
    word = word.translate(bad_chars_translation)
    print(word)
    cursor.query(f"""
    INSERT INTO words (word) VALUES ('{word}')
    """).df()

In [11]:
# check that the words are in the table
cursor.query("""
SELECT * FROM words
""").df()

,_row_id,word
0,1,the
1,2,of
2,3,and
3,4,to
4,5,in
...,...,...
51581,51582,auth
51582,51583,quelled
51583,51584,splinting
51584,51585,empyema


In [12]:
# create a vector index using faiss
# this should take about 10m to run.
cursor.query("""
DROP INDEX IF EXISTS words_vector_idx
""").df()
cursor.query("""
CREATE INDEX words_vector_idx
ON words (EmbedText(words.word))
USING FAISS
""").df()

11-19-2023 01:01:50 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_index:0113] Index words_vector_idx does not exist, therefore cannot be dropped.
/home/oscar/.cache/pypoetry/virtualenvs/words-Cdd64V6x-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


,0
0,Index words_vector_idx successfully added to t...


In [16]:
test_query = "Joy"
cursor.query(f"""
SELECT * FROM words ORDER BY Similarity(EmbedText('{test_query}'), EmbedText(word)) LIMIT 15;
""").df()

,_row_id,word
0,4523,joy
1,16767,joys
2,18096,joyful
3,18967,joyous
4,44434,joyously
5,30361,joyfully
6,14178,happier
7,7077,delight
8,7147,smiling
9,3190,happy


In [17]:
# create a simple flask endpoint to serve the five most similar games
from flask import Flask, request, jsonify
app = Flask(__name__)

# allow cross origin requests from any domain
from flask_cors import CORS
CORS(app)

@app.route('/get_similar_words', methods=['GET'])
def get_similar_games():
    query = request.args.get('query')
    out = cursor.query(f"""
    SELECT * FROM words ORDER BY Similarity(EmbedText('{query}'), EmbedText(word)) LIMIT 15;
    """).df()
    out_recs = out.to_dict(orient="records")
    # replace every url with https with https: (BC EVADB) doesn't let me insert w/ ?
    return jsonify(out_recs)

app.run(host="localhost", port=5002)

# this can be accessed as follows: http://localhost:5000/get_similar_games?query=fast-paced%20action%20game

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5002
Press CTRL+C to quit
127.0.0.1 - - [19/Nov/2023 01:09:31] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [19/Nov/2023 01:09:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Nov/2023 01:09:47] "GET /get_similar_words?query=joy HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2023 01:09:53] "GET /get_similar_words?query=joy HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2023 01:09:58] "GET /get_similar_words?query=joy HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2023 01:10:09] "GET /get_similar_words?query=bad HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2023 01:11:07] "GET /get_similar_words?query=undefined HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2023 01:11:09] "GET /get_similar_words?query=undefined HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2023 01:11:44] "GET /get_similar_words?query=joy HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2023 01:12:05] "GET /get_similar_words?query=smiling HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2023 01:12:12] "GET /get_similar_words?query=grinning HTTP/1.1" 200 -
127.0.0.